In [1]:
from pprint import pprint

from cheryl import Statement, Knows, knows, find_game


%load_ext autoreload
%autoreload 2

In [2]:
domains = [range(1970, 1975), range(5, 10), range(15, 20)]
n_elems = 10

In [3]:
statements = [Statement(author='0', conditions={'0': Knows.no}),
              Statement(author='1', conditions={'1': Knows.no}),
              Statement(author='2', conditions={'2': Knows.no}),
              Statement(author='0', conditions={'0': Knows.no}),
              Statement(author='1', conditions={'1': Knows.no}),
              Statement(author='2', conditions={'2': Knows.no}),
              Statement(author='0', conditions={'0': Knows.no}),
              Statement(author='1', conditions={'1': Knows.no}),
              Statement(author='2', conditions={'2': Knows.no}),
              Statement(author='0', conditions={'0': Knows.yes}),
            ]

game = find_game(domains, n_elems, statements, n_trys=100)
game.get_solution(statements, trace=True)

Before filtering:
     0     	     1     	     2     
|1970 5 19|	|1970 5 19|	|1973 8 16|
|1970 7 18|	|1973 5 18|	|1973 6 18|
|1971 5 19|	|1971 5 19|	|1973 8 18|
|1971 7 19|	|1973 6 18|	|1973 5 18|
|1973 6 18|	|1974 6 18|	|1970 7 18|
|1973 8 18|	|1970 7 18|	|1974 6 18|
|1973 5 18|	|1971 7 19|	|1974 9 18|
|1973 8 16|	|1973 8 18|	|1970 5 19|
|1974 6 18|	|1973 8 16|	|1971 5 19|
|1974 9 18|	|1974 9 18|	|1971 7 19|

After applying statement 1:
     0     	     1     	     2     
|1970 5 19|	|1970 5 19|	|1973 8 16|
|1970 7 18|	|1973 5 18|	|1973 6 18|
|1971 5 19|	|1971 5 19|	|1973 8 18|
|1971 7 19|	|1973 6 18|	|1973 5 18|
|1973 6 18|	|1974 6 18|	|1970 7 18|
|1973 8 18|	|1970 7 18|	|1974 6 18|
|1973 5 18|	|1971 7 19|	|1974 9 18|
|1973 8 16|	|1973 8 18|	|1970 5 19|
|1974 6 18|	|1973 8 16|	|1971 5 19|
|1974 9 18|	|1974 9 18|	|1971 7 19|

After applying statement 2:
     0     	     1     	     2     
|1970 5 19|	|1970 5 19|	|1973 8 16|
|1970 7 18|	|1973 5 18|	|1973 6 18|
|1971 5 19|	|1971 5 19|	

(1970, 5, 19)

In [4]:
statements = [Statement(author='0', conditions={'0': Knows.no, ('1', '2'): Knows.maybe}),
              Statement(author='1', conditions={'1': Knows.no, ('0', '2'): Knows.maybe}),
              Statement(author='2', conditions={'2': Knows.no, ('0', '1'): Knows.maybe}),
              Statement(author='0', conditions={'0': Knows.yes}),
              Statement(author='1', conditions={'1': Knows.yes}),
              Statement(author='2', conditions={'2': Knows.yes}),
             ]

game = find_game(domains, n_elems, statements, n_trys=500)
game.get_solution(statements, trace=True)

Before filtering:
     0     	     1     	     2     
|1970 7 15|	|1973 5 17|	|1970 7 15|
|1970 8 16|	|1970 7 15|	|1971 8 15|
|1970 9 18|	|1973 7 15|	|1973 7 15|
|1971 8 17|	|1974 7 19|	|1974 9 15|
|1971 8 15|	|1970 8 16|	|1970 8 16|
|1972 9 17|	|1971 8 17|	|1972 9 17|
|1973 5 17|	|1971 8 15|	|1971 8 17|
|1973 7 15|	|1972 9 17|	|1973 5 17|
|1974 7 19|	|1974 9 15|	|1970 9 18|
|1974 9 15|	|1970 9 18|	|1974 7 19|

After applying statement 1:
     0     	     1     	     2     
|1970 7 15|	|1973 5 17|	|1970 7 15|
|1970 8 16|	|1970 7 15|	|1973 7 15|
|1970 9 18|	|1973 7 15|	|1974 9 15|
|1973 5 17|	|1974 7 19|	|1970 8 16|
|1973 7 15|	|1970 8 16|	|1973 5 17|
|1974 7 19|	|1974 9 15|	|1970 9 18|
|1974 9 15|	|1970 9 18|	|1974 7 19|

After applying statement 2:
     0     	     1     	     2     
|1970 7 15|	|1973 7 15|	|1973 7 15|
|1970 9 18|	|1970 7 15|	|1970 7 15|
|1973 7 15|	|1974 7 19|	|1974 9 15|
|1974 7 19|	|1974 9 15|	|1970 9 18|
|1974 9 15|	|1970 9 18|	|1974 7 19|

After applying statemen

(1974, 9, 15)

In [5]:
statements = [Statement(author='0', conditions={'0': Knows.no, ('1', '2'): Knows.maybe}),
              Statement(author='1', conditions={'1': Knows.no, ('0', '2'): Knows.maybe}),
              Statement(author='2', conditions={'2': Knows.no, ('0', '1'): Knows.maybe}),
              Statement(author='0', conditions={'0': Knows.yes, '2': Knows.no}),
              Statement(author='1', conditions={'1': Knows.yes, '2': Knows.yes}),
              Statement(author='2', conditions={'2': Knows.yes}),

             ]

game = find_game(domains, n_elems, statements, n_trys=500)
game.get_solution(statements, trace=True)

Before filtering:
     0     	     1     	     2     
|1971 5 17|	|1971 5 17|	|1974 6 15|
|1971 5 16|	|1971 5 16|	|1974 7 16|
|1971 6 18|	|1971 6 18|	|1971 5 16|
|1971 8 17|	|1974 6 15|	|1971 5 17|
|1971 8 18|	|1974 7 16|	|1971 8 17|
|1971 8 19|	|1971 8 17|	|1971 6 18|
|1972 8 18|	|1972 8 18|	|1972 8 18|
|1972 9 18|	|1971 8 18|	|1972 9 18|
|1974 7 16|	|1971 8 19|	|1971 8 18|
|1974 6 15|	|1972 9 18|	|1971 8 19|

After applying statement 1:
     0     	     1     	     2     
|1971 5 17|	|1971 5 17|	|1974 6 15|
|1971 5 16|	|1971 5 16|	|1974 7 16|
|1971 6 18|	|1971 6 18|	|1971 5 16|
|1971 8 17|	|1974 6 15|	|1971 5 17|
|1971 8 18|	|1974 7 16|	|1971 8 17|
|1971 8 19|	|1971 8 17|	|1971 6 18|
|1972 8 18|	|1972 8 18|	|1972 8 18|
|1972 9 18|	|1971 8 18|	|1972 9 18|
|1974 7 16|	|1971 8 19|	|1971 8 18|
|1974 6 15|	|1972 9 18|	|1971 8 19|

After applying statement 2:
     0     	     1     	     2     
|1971 6 18|	|1971 6 18|	|1974 6 15|
|1971 8 17|	|1974 6 15|	|1971 8 17|
|1971 8 18|	|1971 8 17|	

(1972, 8, 18)

In [6]:
statements = [Statement(author='0', conditions={'0': Knows.no, ('1', '2'): Knows.maybe}),
              Statement(author='1', conditions={'1': Knows.no, ('0', '2'): Knows.maybe}),
              Statement(author='2', conditions={'2': Knows.no, ('0', '1'): Knows.maybe}),
              Statement(author='0', conditions={'0': Knows.yes}),
             ]

game = find_game(domains, n_elems, statements, n_trys=500)
game.get_solution(statements, trace=True)

Before filtering:
     0     	     1     	     2     
|1971 5 17|	|1971 5 17|	|1974 6 15|
|1971 5 16|	|1971 5 16|	|1974 7 16|
|1971 6 18|	|1971 6 18|	|1971 5 16|
|1971 8 17|	|1974 6 15|	|1971 5 17|
|1971 8 18|	|1974 7 16|	|1971 8 17|
|1971 8 19|	|1971 8 17|	|1971 6 18|
|1972 8 18|	|1972 8 18|	|1972 8 18|
|1972 9 18|	|1971 8 18|	|1972 9 18|
|1974 7 16|	|1971 8 19|	|1971 8 18|
|1974 6 15|	|1972 9 18|	|1971 8 19|

After applying statement 1:
     0     	     1     	     2     
|1971 5 17|	|1971 5 17|	|1974 6 15|
|1971 5 16|	|1971 5 16|	|1974 7 16|
|1971 6 18|	|1971 6 18|	|1971 5 16|
|1971 8 17|	|1974 6 15|	|1971 5 17|
|1971 8 18|	|1974 7 16|	|1971 8 17|
|1971 8 19|	|1971 8 17|	|1971 6 18|
|1972 8 18|	|1972 8 18|	|1972 8 18|
|1972 9 18|	|1971 8 18|	|1972 9 18|
|1974 7 16|	|1971 8 19|	|1971 8 18|
|1974 6 15|	|1972 9 18|	|1971 8 19|

After applying statement 2:
     0     	     1     	     2     
|1971 6 18|	|1971 6 18|	|1974 6 15|
|1971 8 17|	|1974 6 15|	|1971 8 17|
|1971 8 18|	|1971 8 17|	

(1972, 8, 18)

In [7]:
statements = [
              Statement(author='0', conditions={'0': Knows.no, ('1', '2'): Knows.no}),
              Statement(author='1', conditions={'1': Knows.no, ('0', '2'): Knows.no}),
              Statement(author='2', conditions={'2': Knows.no, ('0', '1'): Knows.no}),
              Statement(author='0', conditions={'0': Knows.no, ('1', '2'): Knows.no}),
              Statement(author='1', conditions={'1': Knows.no, ('0', '2'): Knows.no}),
              Statement(author='0', conditions={'0': Knows.no, ('1', '2'): Knows.no}),
              Statement(author='1', conditions={'1': Knows.no, ('0', '2'): Knows.no}),
              Statement(author='2', conditions={'2': Knows.no, ('0', '1'): Knows.no}),
              Statement(author='0', conditions={'0': Knows.no, ('1', '2'): Knows.no}),
              Statement(author='1', conditions={'1': Knows.no, ('0', '2'): Knows.no}),
              Statement(author='2', conditions={'2': Knows.no, ('0', '1'): Knows.no}),
              Statement(author='0', conditions={'0': Knows.yes}),
             ]

game = find_game(domains, n_elems, statements, n_trys=500)
game.get_solution(statements, trace=True)

Before filtering:
     0     	     1     	     2     
|1970 6 17|	|1974 5 19|	|1974 7 16|
|1971 7 18|	|1974 5 16|	|1974 5 16|
|1971 6 18|	|1972 5 17|	|1970 6 17|
|1972 9 18|	|1970 6 17|	|1974 7 17|
|1972 5 17|	|1971 6 18|	|1972 5 17|
|1972 9 19|	|1971 7 18|	|1971 7 18|
|1974 7 16|	|1974 7 16|	|1971 6 18|
|1974 5 19|	|1974 7 17|	|1972 9 18|
|1974 7 17|	|1972 9 18|	|1974 5 19|
|1974 5 16|	|1972 9 19|	|1972 9 19|

After applying statement 1:
     0     	     1     	     2     
|1971 7 18|	|1974 5 19|	|1974 7 16|
|1971 6 18|	|1974 5 16|	|1974 5 16|
|1972 9 18|	|1972 5 17|	|1974 7 17|
|1972 5 17|	|1971 6 18|	|1972 5 17|
|1972 9 19|	|1971 7 18|	|1971 7 18|
|1974 7 16|	|1974 7 16|	|1971 6 18|
|1974 7 17|	|1974 7 17|	|1972 9 18|
|1974 5 19|	|1972 9 18|	|1974 5 19|
|1974 5 16|	|1972 9 19|	|1972 9 19|

After applying statement 2:
     0     	     1     	     2     
|1971 7 18|	|1974 5 19|	|1974 7 16|
|1972 9 18|	|1974 5 16|	|1974 5 16|
|1972 5 17|	|1972 5 17|	|1974 7 17|
|1972 9 19|	|1971 7 18|	

(1972, 5, 17)

In [8]:
statements = [
              Statement(author='0', conditions={'0': Knows.no, '1': Knows.no, '2': Knows.no}),
              Statement(author='1', conditions={'1': Knows.no, '0': Knows.no, '2': Knows.no}),
              Statement(author='2', conditions={'2': Knows.no, '0': Knows.no, '1': Knows.no}),
              Statement(author='0', conditions={'0': Knows.yes}), 
              Statement(author='1', conditions={'1': Knows.yes}), 
              Statement(author='0', conditions={'2': Knows.yes})
             ]

game = find_game(domains, n_elems, statements, n_trys=1000)
game.get_solution(statements, trace=True)

Before filtering:
     0     	     1     	     2     
|1970 7 19|	|1974 6 19|	|1973 6 15|
|1970 8 18|	|1973 6 15|	|1972 9 15|
|1970 9 18|	|1971 6 16|	|1971 6 16|
|1971 8 17|	|1970 7 19|	|1971 8 17|
|1971 6 16|	|1973 7 19|	|1970 8 18|
|1972 9 15|	|1973 8 19|	|1970 9 18|
|1973 6 15|	|1971 8 17|	|1974 6 19|
|1973 8 19|	|1970 8 18|	|1973 8 19|
|1973 7 19|	|1972 9 15|	|1970 7 19|
|1974 6 19|	|1970 9 18|	|1973 7 19|

After applying statement 1:
     0     	     1     	     2     
|1970 7 19|	|1973 6 15|	|1973 6 15|
|1970 8 18|	|1970 7 19|	|1970 8 18|
|1970 9 18|	|1973 7 19|	|1970 9 18|
|1973 6 15|	|1973 8 19|	|1973 8 19|
|1973 8 19|	|1970 8 18|	|1970 7 19|
|1973 7 19|	|1970 9 18|	|1973 7 19|

After applying statement 2:
     0     	     1     	     2     
|1970 8 18|	|1973 7 19|	|1970 8 18|
|1970 7 19|	|1970 7 19|	|1973 8 19|
|1973 8 19|	|1970 8 18|	|1973 7 19|
|1973 7 19|	|1973 8 19|	|1970 7 19|

After applying statement 3:
     0     	     1     	     2     
|1970 7 19|	|1973 7 19|	|1973 8

(1970, 7, 19)